In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load the CSVs
# Load the data
vehicle_df = pd.read_csv('/home/arjunrao/mini_proj_6/train_model/vehicle_data.csv')
network_df = pd.read_csv('/home/arjunrao/mini_proj_6/train_model/network_data.csv')

# Merge on `time_step` and `ap_id`
vehicle_df['ap_id'] = vehicle_df['ap_id'].astype(str)
network_df['ap_id'] = network_df['ap_id'].str.extract(r'(\d+)').astype(str)  # clean `ap_1` to `1`
merged_df = pd.merge(network_df, vehicle_df, on=['time_step', 'ap_id'])

# Select relevant features
features = ['avg_packet_rate', 'avg_latency', 'bandwidth_usage', 'speed', 'acceleration', 'active_nodes']
merged_df = merged_df[["time_step", "ap_id"] + features]
# Normalize


scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(merged_df[features])

# Create sequences
sequence_length = 10
X, y = [], []
for i in range(len(df_scaled) - sequence_length):
    X.append(df_scaled[i:i + sequence_length])
    y.append(df_scaled[i + sequence_length])  

X = np.array(X)
y = np.array(y)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

# Save test data
np.save("X_test.npy", X_test)
np.save("y_test.npy", y_test)

print("Saved X_test.npy and y_test.npy with shape:", X_test.shape)


Saved X_test.npy and y_test.npy with shape: (452, 10, 6)


In [3]:
from tensorflow.keras.models import load_model
model = load_model("network_congestion_lstm.h5")
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 128)           69120     
                                                                 
 dropout (Dropout)           (None, 10, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 6)                 390       
                                                                 
Total params: 118918 (464.52 KB)
Trainable params: 118918 (464.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
import time
import socket
import json
import numpy as np
from tensorflow.keras.models import load_model

# Load LSTM model
model = load_model("network_congestion_lstm.h5")

# Load test data
X_test = np.load("X_test.npy")

# Feature labels used (same order as during training)
feature_labels = ['avg_packet_rate', 'avg_latency', 'bandwidth_usage', 'speed', 'acceleration', 'active_nodes']

# Set up UDP
controller_ip = "127.0.0.1"
controller_port = 9999
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

print("Starting ML prediction sender loop...")
i = 0
while True:
    all_ap_predictions = []

    for ap_id in range(1, 11):
        if i >= X_test.shape[0]:
            i = 0

        x_input = X_test[i:i+1]
        prediction = model.predict(x_input, verbose=0)[0]  # Shape (6,)

        predicted_features = {
            feature_labels[j]: round(float(prediction[j]), 4)
            for j in range(len(feature_labels))
        }

        ap_data = {
            "ap_id": ap_id,
            "predicted_features": predicted_features
        }

        all_ap_predictions.append(ap_data)
        i += 1

    payload = {
        "type": "batch",
        "data": all_ap_predictions
    }

    sock.sendto(json.dumps(payload).encode(), (controller_ip, controller_port))
    print(f"[{time.strftime('%H:%M:%S')}] Sent batch prediction.")

    time.sleep(5)


2025-05-08 15:46:37.534200: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-08 15:46:37.536958: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-08 15:46:37.594276: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-08 15:46:37.595474: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-08 15:46:38.537094: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Starting ML prediction sender loop...
[15:46:42] Sent batch prediction.
[15:46:47] Sent batch prediction.
[15:46:53] Sent batch prediction.
[15:46:59] Sent batch prediction.
[15:47:04] Sent batch prediction.
[15:47:10] Sent batch prediction.
[15:47:15] Sent batch prediction.
[15:47:21] Sent batch prediction.
[15:47:27] Sent batch prediction.


KeyboardInterrupt: 

In [8]:
import time
import socket
import json
import numpy as np
from tensorflow.keras.models import load_model

# Load LSTM model and test data
model = load_model("network_congestion_lstm.h5")
X_test = np.load("X_test.npy")

feature_labels = ['avg_packet_rate', 'avg_latency', 'bandwidth_usage', 'speed', 'acceleration', 'active_nodes']

# MAC addresses for APs (1-10)
ap_mac_map = {
    i: f"00:00:00:00:00:{i:02X}" for i in range(1, 11)
}

# UDP setup
controller_ip = "127.0.0.1"
controller_port = 9999
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

print("Starting ML prediction sender loop...")
i = 0
while True:
    all_ap_predictions = []

    for ap_id in range(1, 11):
        if i >= X_test.shape[0]:
            i = 0

        x_input = X_test[i:i+1]
        prediction = model.predict(x_input, verbose=0)[0]  # Shape (6,)

        predicted_features = {
            feature_labels[j]: round(float(prediction[j]), 4)
            for j in range(len(feature_labels))
        }

        # Compute simple congestion score (customize as needed)
        congestion_score = round(
            0.4 * predicted_features['avg_packet_rate'] +
            0.3 * predicted_features['avg_latency'] +
            0.3 * predicted_features['bandwidth_usage'],
            4
        )

        ap_data = {
            "ap_id": ap_id,
            "mac": ap_mac_map[ap_id],
            "congestion_score": congestion_score,
            **predicted_features
        }

        all_ap_predictions.append(ap_data)
        i += 1

    payload = {
        "type": "batch",
        "data": all_ap_predictions
    }

    sock.sendto(json.dumps(payload).encode(), (controller_ip, controller_port))
    print(f"[{time.strftime('%H:%M:%S')}] Sent batch prediction.")

    time.sleep(5)


Starting ML prediction sender loop...
[13:18:37] Sent batch prediction.
[13:18:42] Sent batch prediction.
[13:18:48] Sent batch prediction.
[13:18:53] Sent batch prediction.
[13:18:58] Sent batch prediction.
[13:19:04] Sent batch prediction.
[13:19:09] Sent batch prediction.
[13:19:14] Sent batch prediction.
[13:19:20] Sent batch prediction.
[13:19:25] Sent batch prediction.
[13:19:30] Sent batch prediction.
[13:19:36] Sent batch prediction.
[13:19:41] Sent batch prediction.
[13:19:46] Sent batch prediction.
[13:19:52] Sent batch prediction.
[13:19:57] Sent batch prediction.
[13:20:02] Sent batch prediction.
[13:20:08] Sent batch prediction.
[13:20:13] Sent batch prediction.
[13:20:18] Sent batch prediction.
[13:20:24] Sent batch prediction.
[13:20:29] Sent batch prediction.
[13:20:34] Sent batch prediction.
[13:20:40] Sent batch prediction.
[13:20:45] Sent batch prediction.
[13:20:50] Sent batch prediction.
[13:20:56] Sent batch prediction.
[13:21:01] Sent batch prediction.
[13:21:07]

KeyboardInterrupt: 

In [5]:
print(json.dumps(payload))  # Log the payload to see what is being sent
sock.sendto(json.dumps(payload).encode(), (controller_ip, controller_port))


{"type": "batch", "data": [{"ap_id": 1, "mac": "00:00:00:00:00:01", "congestion_score": 0.2959, "avg_packet_rate": 0.312, "avg_latency": 0.1938, "bandwidth_usage": 0.3764, "speed": 0.4976, "acceleration": 0.5483, "active_nodes": 0.6054}, {"ap_id": 2, "mac": "00:00:00:00:00:02", "congestion_score": 0.4786, "avg_packet_rate": 0.5296, "avg_latency": 0.3841, "bandwidth_usage": 0.505, "speed": 0.4943, "acceleration": 0.5578, "active_nodes": 0.8528}, {"ap_id": 3, "mac": "00:00:00:00:00:03", "congestion_score": 0.4803, "avg_packet_rate": 0.5042, "avg_latency": 0.4552, "bandwidth_usage": 0.4734, "speed": 0.5007, "acceleration": 0.5643, "active_nodes": 0.3438}, {"ap_id": 4, "mac": "00:00:00:00:00:04", "congestion_score": 0.5901, "avg_packet_rate": 0.7303, "avg_latency": 0.4, "bandwidth_usage": 0.5933, "speed": 0.5009, "acceleration": 0.5663, "active_nodes": 0.612}, {"ap_id": 5, "mac": "00:00:00:00:00:05", "congestion_score": 0.4738, "avg_packet_rate": 0.3654, "avg_latency": 0.4826, "bandwidth_u

2141

In [6]:
print(predicted_features)  # Log to check the values


{'avg_packet_rate': 0.4864, 'avg_latency': 0.5343, 'bandwidth_usage': 0.4662, 'speed': 0.4988, 'acceleration': 0.5653, 'active_nodes': 0.3431}
